In [1]:
import torch
import numpy as np
#import scipy.io
import argparse
from pinn.neural_net import MLP
from pinn.get_points import normalize
from pinn.util import log
from pathlib import Path
import matplotlib.pyplot as plt
from pinn.evaluate import get_pars, evaluate, load_data, get_residuals, get_loss
import pandas as pd
import glob
import re

2022-05-31 18:43:39,756  29 MainThread INFO     numexpr.utils:147 - _init_num_threads: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2022-05-31 18:43:39,757  29 MainThread INFO     numexpr.utils:159 - _init_num_threads: NumExpr defaulting to 8 threads.


In [2]:
#cases = ['model_nf1000_nb1000_ns0_fn1-0-0_fs0-1-1_shuffle0.pt',
#         'model_nf1000_nb1000_ns0_fn1-0-0_fs0-1-1_shuffle1.pt',
#         'model_nf3000_nb1000_ns0_fn1-0-0_fs0-1-1_shuffle0.pt',
#         'model_nf3000_nb1000_ns0_fn1-0-0_fs0-1-1_shuffle1.pt',
#         'model_nf10000_nb1000_ns0_fn1-0-0_fs0-1-1_shuffle0.pt',
#         'model_nf10000_nb1000_ns0_fn1-0-0_fs0-1-1_shuffle1.pt',
#         'model_nf30000_nb1000_ns0_fn1-0-0_fs0-1-1_shuffle0.pt',
#         'model_nf30000_nb1000_ns0_fn1-0-0_fs0-1-1_shuffle1.pt',
#         'model_nf100000_nb1000_ns0_fn1-0-0_fs0-1-1_shuffle0.pt',
#         'model_nf100000_nb1000_ns0_fn1-0-0_fs0-1-1_shuffle1.pt',
#         'model_nf10000_nb1000_ns0_fn1-10-5_fs0-2-1_shuffle0.pt',
#         'model_nf10000_nb1000_ns0_fn1-20-10_fs0-2-1_shuffle0.pt'
#         ]

cases = [f for f in glob.glob("models/*.pt")]

In [3]:
def summarize(case):
    nt = 51
    nx = 101
    ny = 11

    t = np.linspace(0,12*3600,nt)
    x = np.linspace(0,10000,nx)
    y = np.linspace(0,250,ny)


    [E, U, V, t_grid, x_grid, y_grid, X] = evaluate(t,x,y,case)

    [r1, r2, r3, t_grid, x_grid, y_grid, X] = get_residuals(t,x,y,case)
    R_rms = np.sqrt(np.sum(r1**2) + np.sum(r2**2) + np.sum(r3**2))/(12*3600*np.sqrt(nt*nx*ny))
    R_abs = (np.sum(np.abs(r1)) + np.sum(np.abs(r2)) + np.sum(np.abs(r3)))/(12*3600*nt*nx*ny)

    m = torch.load(case)

    log_loss = m['loss']
    time = m['time']
    memory = m['memory']
    loss = log_loss[-1,0]
    iter = log_loss.shape[0]
    U_max = np.max(U)

    #return {'case':case,'t':t,'x':x,'y':y,'E':E,'U':U,'V':V,'R_rms':R_rms,'R_abs':R_abs,'loss':log_loss,'time':time,'memory':memory}
    return {'case':case,'R_rms':R_rms,'R_abs':R_abs,'iter':iter,'loss':loss,'log_loss':log_loss,'time':time,'memory':memory,'U_max':U_max}

In [4]:
S = pd.DataFrame.from_dict([summarize(case) for case in cases])

In [5]:
pd.set_option('display.max_colwidth', 120)
S[['case','memory','time','R_rms','R_abs','loss','iter','U_max']]

,case,memory,time,R_rms,R_abs,loss,iter,U_max
0,models/model_nf3000_nb1000_ns0nf_delta10000_fn1-0-0_fs0-1-1_MLPRes_2x2x20_shuffle0_seed1_adam_lr0.001_loss_l1.pt,102.726074,8427.299298,0.000112,0.000090,0.000067,49999,0.082461
1,models/model_nf30000_nb1000_ns0nf_delta10000_fn1-0-0_fs0-1-1_MLPRes_2x2x20_shuffle1_seed1_adam_lr0.001_loss_l1.pt,982.700195,9773.714826,0.000084,0.000091,0.000056,49999,0.124976
2,models/model_nf3000_nb1000_ns0nf_delta10000_fn1-0-0_fs0-1-1_MLPRes_2x2x20_shuffle0_seed2_adam_lr0.001_loss_l1.pt,102.726074,8415.405435,0.000093,0.000088,0.000069,49999,0.074140
3,models/model_nf1000_nb1000_ns0nf_delta10000_fn1-0-0_fs0-1-1_MLPRes_2x2x20_shuffle0_seed2_adam_lr0.001_loss_l1.pt,37.545898,8419.702273,0.000213,0.000118,0.000067,49999,0.053187
4,models/model_nf300_nb1000_ns0nf_delta10000_fn1-0-0_fs0-1-1_MLPRes_2x2x20_shuffle0_seed1_adam_lr0.001_loss_l1.pt,14.750000,6999.211867,0.000283,0.000139,0.000051,49999,0.025798
5,models/model_nf3000_nb1000_ns0nf_delta10000_fn1-0-0_fs0-1-1_MLPRes_2x2x20_shuffle1_seed1_adam_lr0.001_loss_l1.pt,102.726074,8780.512274,0.000090,0.000088,0.000066,49999,0.089951
6,models/model_nf1000_nb1000_ns0nf_delta10000_fn1-0-0_fs0-1-1_MLPRes_2x2x20_shuffle1_seed1_adam_lr0.001_loss_l1.pt,37.545898,8858.075607,0.000131,0.000103,0.000086,49999,0.064354
7,models/model_nf300_nb1000_ns0nf_delta10000_fn1-0-0_fs0-1-1_MLPRes_2x2x20_shuffle1_seed1_adam_lr0.001_loss_l1.pt,14.750000,7536.853683,0.000128,0.000099,0.000093,49999,0.034265
8,models/model_nf1000_nb1000_ns0nf_delta10000_fn1-0-0_fs0-1-1_MLPRes_2x2x20_shuffle0_seed1_adam_lr0.001_loss_l1.pt,37.545898,8411.347518,0.000190,0.000113,0.000072,49999,0.045581
9,models/model_nf300_nb1000_ns0nf_delta10000_fn1-0-0_fs0-1-1_MLPRes_2x2x20_shuffle0_seed2_adam_lr0.001_loss_l1.pt,14.750000,6984.302615,0.000280,0.000145,0.000060,49999,0.024662


In [6]:
R = pd.DataFrame(columns=['nf','shuffle','R_rms','R_abs','R_rms_min','R_rms_max','R_abs_min','R_abs_max','time','time_min','time_max','memory','n_runs'])

for ind, l in S.iterrows():
    nf = int(re.search('nf(\d+)', l['case']).group(1))
    shuffle = int(re.search('shuffle(\d+)', l['case']).group(1))

    known = -1
    for i, (NF, SHUFFLE) in enumerate(zip(R.nf.tolist(), R.shuffle.tolist())):
        if nf == NF and shuffle == SHUFFLE:
            known = i
    
    if known < 0:
        R = R.append({'nf':nf,'shuffle':shuffle,
                    'R_rms':l['R_rms'],'R_abs':l['R_abs'],
                    'R_rms_min':l['R_rms'],'R_rms_max':l['R_rms'],
                    'R_abs_min':l['R_abs'],'R_abs_max':l['R_abs'],
                    'time':l['time'],'time_min':l['time'],'time_max':l['time'],
                    'memory':l['memory'],'n_runs':1},ignore_index=True)
    else:
        current = R.loc[known]
        current.R_rms = (current.n_runs * current.R_rms + l.R_rms)/(current.n_runs+1)
        nr = R.loc[known,'n_runs']
        R.loc[known,'n_runs'] = nr + 1
        R.loc[known,'R_rms'] = (R.loc[known,'R_rms']*nr + l['R_rms'])/(nr+1)
        R.loc[known,'R_rms_min'] = min(R.loc[known,'R_rms_min'], l['R_rms'])
        R.loc[known,'R_rms_max'] = max(R.loc[known,'R_rms_max'], l['R_rms'])
        R.loc[known,'R_abs'] = (R.loc[known,'R_abs']*nr + l['R_abs'])/(nr+1)
        R.loc[known,'R_abs_min'] = min(R.loc[known,'R_abs_min'], l['R_abs'])
        R.loc[known,'R_abs_max'] = max(R.loc[known,'R_abs_max'], l['R_abs'])
        R.loc[known,'time'] = (R.loc[known,'time']*nr + l['time'])/(nr+1)
        R.loc[known,'time_min'] = min(R.loc[known,'time_min'], l['time'])
        R.loc[known,'time_max'] = max(R.loc[known,'time_max'], l['time'])

R =R.sort_values(['nf','shuffle'])

/tmp/ipykernel_29110/921812561.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  R = R.append({'nf':nf,'shuffle':shuffle,
/tmp/ipykernel_29110/921812561.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  R = R.append({'nf':nf,'shuffle':shuffle,
/tmp/ipykernel_29110/921812561.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  R = R.append({'nf':nf,'shuffle':shuffle,
/tmp/ipykernel_29110/921812561.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  R = R.append({'nf':nf,'shuffle':shuffle,
/tmp/ipykernel_29110/921812561.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. 

In [7]:
R

,nf,shuffle,R_rms,R_abs,R_rms_min,R_rms_max,R_abs_min,R_abs_max,time,time_min,time_max,memory,n_runs
3,300.0,0.0,0.000281,0.000142,0.00028,0.000283,0.000139,0.000145,6991.757241,6984.302615,6999.211867,14.75,2.0
6,300.0,1.0,0.000128,0.000099,0.000128,0.000128,0.000099,0.000099,7536.853683,7536.853683,7536.853683,14.75,1.0
2,1000.0,0.0,0.000196,0.000116,0.00019,0.000213,0.000113,0.000118,8415.524896,8411.347518,8419.702273,37.545898,2.0
5,1000.0,1.0,0.000131,0.000103,0.000131,0.000131,0.000103,0.000103,8858.075607,8858.075607,8858.075607,37.545898,1.0
0,3000.0,0.0,0.000098,0.000089,0.000093,0.000112,0.000088,0.00009,8421.352367,8415.405435,8427.299298,102.726074,2.0
4,3000.0,1.0,0.00009,0.000088,0.00009,0.00009,0.000088,0.000088,8780.512274,8780.512274,8780.512274,102.726074,1.0
8,30000.0,0.0,0.000083,0.000089,0.000083,0.000083,0.000089,0.000089,9693.894631,9693.894631,9693.894631,982.700195,1.0
1,30000.0,1.0,0.000084,0.000091,0.000084,0.000084,0.000091,0.000091,9773.714826,9773.714826,9773.714826,982.700195,1.0
7,100000.0,0.0,0.000081,0.000086,0.000081,0.000081,0.000086,0.000086,16811.40537,16811.40537,16811.40537,3275.965332,1.0
